In [95]:
import pandas as pd

# Try reading the CSV, ignoring bad lines
df = pd.read_csv('data/reviews_1250-end.csv', on_bad_lines='skip')

# Check if any rows were skipped
if df.shape[0] < 150387:  # Assuming the file should have at least 150387 rows
    print("Warning: Some rows were skipped due to errors.")

print(df.head())

   Unnamed: 0    author_id  rating  is_recommended  helpfulness  \
0           0   1945004256       5             1.0     0.000000   
1           1   5478482359       3             1.0     0.333333   
2           2  29002209922       5             1.0     1.000000   
3           3   7391078463       5             1.0     1.000000   
4           4   1766313888       5             1.0     1.000000   

   total_feedback_count  total_neg_feedback_count  total_pos_feedback_count  \
0                     2                         2                         0   
1                     3                         2                         1   
2                     2                         0                         2   
3                     2                         0                         2   
4                    13                         0                        13   

  submission_time                                        review_text  \
0      2022-12-10  I absolutely L-O-V-E this oil. 

C:\Users\Nicolas\AppData\Local\Temp/ipykernel_13600/271379195.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/reviews_1250-end.csv', on_bad_lines='skip')


In [96]:
df1=df.copy()

In [97]:
df = df[['author_id', 'rating', 'product_id']]

In [98]:
#df = df.iloc[:len(df)//7]

In [99]:
df.shape

(49977, 3)

In [154]:
df['author_id'] = pd.factorize(df['author_id'])[0] + 1


In [101]:
df.head()

,author_id,rating,product_id
0,1,5,P379064
1,2,3,P379064
2,3,5,P379064
3,4,5,P379064
4,5,5,P379064


In [102]:
df.dtypes

author_id      int64
rating         int64
product_id    object
dtype: object

In [103]:
df['author_id'] = df['author_id'].astype(str)

# Filter out rows where author_id starts with 'order'
df = df[~df['author_id'].str.startswith(('order'))]
df['product_id'] = df['product_id'].apply(lambda x: x[1:])

In [104]:

df['author_id'] = df['author_id'].astype('int64')
df['product_id'] = df['product_id'].astype('int64')


In [105]:
df.dtypes

author_id     int64
rating        int64
product_id    int64
dtype: object

In [106]:
#!pip install pyspark

In [107]:

from pyspark.sql.functions import col, explode
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

In [108]:

num_ratings = df["rating"].count()
num_users = df["author_id"].nunique()
num_movies = df["product_id"].nunique()

denominator = num_users * num_movies

sparsity = (1.0 - (num_ratings * 1.0) / denominator) * 100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.89% empty.


In [109]:
from implicit.als import AlternatingLeastSquares
model = AlternatingLeastSquares(factors=190, regularization=0.01, iterations=10)

In [147]:

from scipy.sparse import csr_matrix
sparse_matrix = csr_matrix((df['rating'], (df['author_id'], df['product_id'])))

In [149]:

model.fit(sparse_matrix.T)

c:\Users\Nicolas\AppData\Local\Programs\Python\Python310\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.004002571105957031 seconds
  warnings.warn(
100%|██████████| 15/15 [00:51<00:00,  3.41s/it]


In [113]:
#pip install implicit


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Nicolas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [152]:
ecommendations = model.recommend(10, sparse_matrix, N=10)

ValueError: user_items must contain 1 row for every user in userids

## Spark

In [177]:
!pip install pyspark

You should consider upgrading via the 'C:\Users\Nicolas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [180]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

In [181]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [182]:
df_spark = spark.createDataFrame(df)

In [183]:
# Create test and train set
(train, test) = df_spark.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="author_id", itemCol="product_id", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [184]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \




In [185]:
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


In [186]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_a52cac1b3cc3


In [187]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

Py4JJavaError: An error occurred while calling o2635.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 15.0 failed 1 times, most recent failure: Lost task 1.0 in stage 15.0 (TID 31) (192.168.0.244 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RD

In [ ]:

# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())